In [1]:
import requests
import pandas as pd

DEST_FOLDER = "data/"
LIST_FILE = DEST_FOLDER + "concession_list.csv"

/Users/ysuzuki2/Library/CloudStorage/OneDrive-WBG/Documents/GitHub/Romania-Procurement/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Get the list of relevant documents 

a. Business Type: Construction

b. Dates : 2022/01/01 - 2025/10/6

c. Document types - Concessions

In [2]:
# Send a POST request to a sample API endpoint and save the response as CSV

import json

url = "https://www.e-licitatie.ro/api-pub/NoticeCommon/GetCNoticeList/"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://e-licitatie.ro/pub/notices/contract-notices/list/0/0",
}
payloads = {
    "sysNoticeTypeIds": [7], # 7 for Concessions
    "sortProperties": [],
    "pageSize": 1200,
    "hasUnansweredQuestions": False,
    "startPublicationDate": "2022-01-01T20:02:15.771Z",
    "startTenderReceiptDeadline": "2022-01-01T20:02:15.771Z",
    "sysProcedureStateId": None, # all states e.g in progress, cancelled, etc.
    "pageIndex": 0,
    "cPVCategoryId": 2 # 2 for Construction
}


response = requests.post(url, json=payloads, headers=headers)

df = pd.DataFrame(json.loads(response.content)['items'])
df.to_csv(LIST_FILE, index=False)
print(f"Response saved as {LIST_FILE}")

Response saved as data/concession_list.csv


In [18]:
import pandas as pd
contract_list = pd.read_csv(LIST_FILE)

In [11]:
general_data 


{'message': 'Access Denied: Referrer cannot be null.'}

## Get the information for each of the contract

In [12]:
import json
import pandas as pd
import requests
contract_list = pd.read_csv(LIST_FILE)
contract_list.sort_values(by='cNoticeId', inplace=True)
# List of document_ids to loop through
results = []
batch_size = 100  # Change as needed
start_index = 0  
for idx, row in enumerate(contract_list[start_index:].itertuples(), start=start_index):
    contract_id = row.cNoticeId
    document_type = row.sysNoticeTypeId

    row_data = {"document_id": contract_id}
    
    # Get general info to extract dfNoticeId and save general data
    general_info_url = f'https://e-licitatie.ro/api-pub/comboPub/getNoticeGeneralInfo/?noticeId={contract_id}&sysNoticeTypeId={document_type}'
    try:
        general_response = requests.get(general_info_url, headers={
                "Referer": f"https://www.e-licitatie.ro/pub/notices/c-notice/v2/view/{contract_id}"
            },)
        general_data = json.loads(general_response.content.decode("utf-8"))
        dfNoticeId = general_data.get('dfNoticeId', 111)  # Default to 111 if not found
        print(dfNoticeId)
        # Add general data to row_data with "general_" prefix
        for k, v in general_data.items():
            row_data[f"general_{k}"] = v
            
    except Exception as e:
        print(f"Failed to get general info for {contract_id}: {e}")
        dfNoticeId = 111  # Fallback to default
    
    urls = {
        "section1": f"https://www.e-licitatie.ro/api-pub/NoticeCommon/GetSection1View/?initNoticeId={contract_id}&sysNoticeTypeId={document_type}",
        "section2": f"https://www.e-licitatie.ro/api-pub/NoticeCommon/GetSection21View/?dfNoticeId={dfNoticeId}&initNoticeId={contract_id}&sysNoticeTypeId={document_type}",
        "section3": f"https://www.e-licitatie.ro/api-pub/NoticeCommon/GetSection3View/?initNoticeId={contract_id}&sysNoticeTypeId={document_type}",
        "section4": f"https://www.e-licitatie.ro/api-pub/NoticeCommon/GetSection4View/?initNoticeId={contract_id}&sysNoticeTypeId={document_type}",
        "section6": f"https://www.e-licitatie.ro/api-pub/NoticeCommon/GetSection6View/?initNoticeId={contract_id}&sysNoticeTypeId={document_type}",
    }
    for section, url in urls.items():
        response = requests.get(
            url,
            headers={
                "Referer": f"https://www.e-licitatie.ro/pub/notices/c-notice/v2/view/{contract_id}"
            },
        )
        try:
            data = json.loads(response.content.decode("utf-8"))
            # Flatten each key with section identifier
            for k, v in data.items():
                row_data[f"{section}_{k}"] = v
        except Exception as e:
            print(f"Failed to parse JSON for {section} of {contract_id}: {e}")
    results.append(row_data)

    # Save every batch_size rows to a distinct file
    if (idx + 1) % batch_size == 0 or (idx + 1) == len(contract_list):
        batch_df = pd.DataFrame(results)
        batch_file = f"{DEST_FOLDER}consolidated_concession_batch_{idx // batch_size + 1}.csv"
        batch_df.to_csv(batch_file, index=False)
        print(f"Saved batch to {batch_file}")
        results = []  # Reset for next batch

100215199
100221134
100223552
100240283
100252116
100258272
100272838
100299648
100303024
100353273
100349235
100368147
Saved batch to data/consolidated_concession_batch_1.csv


In [56]:
response = requests.get(
    url,
    headers={
        "Referer": f"https://www.e-licitatie.ro/pub/notices/c-notice/v2/view/{contract_id}"
    },
)

In [54]:
response.content

b'{"personalSituation":"Cerinta 1 : 1. Ofertantii, ter\xc8\x9bii sus\xc8\x9binatori si subcontractantii nu trebuie sa se regaseasca:\\nModalitatea de indeplinire: Se va completa DUAE de c\xc4\x83tre operatorii economici participan\xc8\x9bi la procedura de atribuire cu informa\xc8\x9biile aferente situa\xc8\x9biei lor. Documentele justificative care probeaz\xc4\x83 \xc3\xaendeplinirea celor asumate prin completarea DUAE urmeaz\xc4\x83 a fi prezentate, la solicitarea autorit\xc4\x83\xc8\x9bii contractante, doar de c\xc4\x83tre ofertantul clasat pe locul I \xc3\xaen clasamentul intermediar \xc3\xaentocmit la finalizarea evalu\xc4\x83rii ofertelor. Aceste documente sunt: \\n- Certificate constatatoare/certificate de atestare fiscala, privind lipsa datoriilor cu privire la plata impozitelor, taxelor sau a contribu\xc8\x9biilor la bugetul general consolidat valabile la momentul prezent\xc4\x83rii.        \\n - Certificat de atestare fiscal\xc4\x83 privind impozitele \xc8\x99i taxele locale \